This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '***********'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json", API_KEY)
print(r.json()['dataset_data']['column_names'])
print(r.json()['dataset_data']['data'][9])

[u'Date', u'Open', u'High', u'Low', u'Close', u'Change', u'Traded Volume', u'Turnover', u'Last Price of the Day', u'Daily Traded Units', u'Daily Turnover']
[u'2019-05-15', None, 87.35, 85.1, 86.65, None, 145855.0, 12611298.0, None, None, None]


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
#print(r.json())

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Read JSON data into a dictionary. Remove one layer of nesting
afxdict= dict(r.json()['dataset_data'])
#Save column names for a future dataframe
column_names= [afxdict['column_names'][1:]]
#initialize empty dictionary, data
data={}
#Iterate through list of lists in order to populate data with 2017 data
for line in afxdict['data']:
    #Use list indexing and string method to specify year
    if line[0].startswith('2017'):
        #Populate data with date keys corresponding to lists of stock data
        data[line[0]]= line[1:]
#Convert to dataFrame for easier use. Unfornetly, the dates have become the columns
df=pd.DataFrame(data)
#perform a double unstack to move the columns to the index
df=df.unstack(level=df.index).unstack()
#Name columns
df.columns=column_names

#Find the Max and Min
maxOpen=float(df.Open.max())
minOpen=float(df.Open.min())



print('The highest opening price of 2017 was $'+ str(maxOpen)+ ' and the lowest was $'+ str(minOpen))

The highest opening price of 2017 was $53.11 and the lowest was $34.0


In [6]:
#Define empty dictionary, swing
swing= {}
#Iterate through list of lists in order to populate data with 2017 data
for line in afxdict['data']:
    if line[0].startswith('2017'):
        #Find the difference between the daily high and daily low
        swing[line[0]]= line[2]-line[3]
                     
#Convert to Series for easier use
swing=pd.Series(swing)
#Determine the largest difference
maxDif =float(swing.max())
print('The largest swing in the stock price was $'+ str(maxDif)) +' on May 11th'

The largest swing in the stock price was $2.81 on May 11th


In [7]:
#Initialize empty dictionary, difference
difference= {}
#Initialize place holder variable
hold=0
#Iterate backwards because list have most recent dates first 
for line in  afxdict['data'][::-1]:
    #Only want data for 2017 but to get the difference on 2017 we need the last data point of 2016
    if line[0].startswith('2017') or line[0]=='2016-12-30':
        #Populate difference with dates as keys and the change in closing prices as values
        difference[line[0]]=line[4] - hold
        #Place value in hold so it can be use on next iteration
        hold=line[4]
#Convert to series for easier us
difference=pd.Series(difference)
#Use absolute value to ensure negative value are included in max
difference=abs(difference)
#Add to DataFrame to eliminate 2016 value
df['difference']=difference
#Find Max 
maxDif =float(df['difference'].max())
print('The largest difference in closing price from one day to the next was $'+ str(maxDif)+' from August 8th to August 9th')

The largest difference in closing price from one day to the next was $2.56 from August 8th to August 9th


In [8]:
#Initialize empty dictionary, volume
volume={}
#Iterate through list of lists
for line in afxdict['data']:
    #Only 2017 values
    if line[0].startswith('2017'):
        #Populate volume with the volume portion of the list
        volume[line[0]]= line[6]
#Convert to Series
volume=pd.Series(volume)
#Find average
average= volume.mean()
print('The average trading volume in the year 2017 was '+ str(average))

The average trading volume in the year 2017 was 89124.33725490196


In [9]:
#Initialize empty dictionary, volumeMed
volumeMed={}
#Iterate through list of lists
for line in afxdict['data']:
    #Only 2017 values
    if line[0].startswith('2017'):
        #Populate volumeMed with the volume portion of the list
        volumeMed[line[0]]= line[6]
#Convert to Series        
volumeMed=pd.Series(volumeMed)
#Find Median
med= volumeMed.median()
print('The median trading volume in the year 2017 was '+ str(med))

The median trading volume in the year 2017 was 76286.0
